In [ ]:
# Using tensorflow python 37 Environment

In [1]:
import logging
from typing import Optional, Tuple

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

from PyEMD.splines import akima, cubic_spline_3pts
from PyEMD.utils import get_timeline

import pylab as plt
import warnings
import time
from pandas import concat
from pandas import DataFrame
from numpy import array

In [2]:
from math import sqrt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation, Dropout, Flatten, Conv1D, MaxPooling1D, ConvLSTM2D
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

In [ ]:
# 01. Upload Data

In [3]:
df_Br = pd.read_csv('Br_close.csv')
df_Br.set_index(pd.DatetimeIndex(df_Br['Date']), inplace=True)
df_Br.drop(['Date'],axis=1,inplace=True)
df_Br.head()

,Close
Date,
2000-01-04,24.39
2000-01-05,23.73
2000-01-06,23.62
2000-01-07,23.09
2000-01-10,23.73


In [ ]:
# 02. Knowing How Much IMF in each Date

In [ ]:
# Upload Data Jumlah Kol

In [4]:
df_Kol = pd.read_csv('Jumlah_IMF_EMD.csv')
df_Kol.set_index(pd.DatetimeIndex(df_Kol['Date']), inplace=True)
df_Kol.drop(['Date'],axis=1,inplace=True)
df_Kol.head()

,Jum_Kol
Date,
2018-11-27,10
2018-11-28,11
2018-11-29,10
2018-11-30,11
2018-12-03,11


In [ ]:
# 03. Univariate CNN LSTM

In [ ]:
# Prediksi data IMF 1 semua

In [5]:
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [6]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [7]:
def series_to_supervised(data, n_in, n_out=30):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

In [8]:
def model_predict(model, history, config):
    # unpack config
    n_seq, n_steps, _, _, _, _, _ = config
    n_input = n_seq * n_steps
    # prepare data
    x_input = array(history[-n_input:]).reshape((1, n_seq, n_steps, 1))
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return yhat[0]

In [9]:
# Untuk Horizon lebih dari satu
def split_sequence2(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Testing 2 Hor :

In [10]:
lim = 900
iIMF = 1 # Letak IMF yang diambil
n_test = 30 # horizon = 2
pred_iIMF = []

In [11]:
pred_all = []
n_features = 1 # klo horizon ini tetap satu saja

In [12]:
# Parameter and Define CNN LSTM Model 
n_seq = 3
n_steps = 15
n_steps_out = 30

n_input2 = n_seq * n_steps

n_filters = 64
n_kernel = 3
n_nodes = 100
n_epochs = 200
n_batch = 100

In [13]:
# pred_iIMF

[0.40960658, 0.1013519, -0.31524065, -0.06470162]

In [ ]:
# Ini Proses Sekaligus semua IMF

In [24]:
start = time.time()
lim = 900
n_test = 30 # horizon = 2
# cfg = [24, 500, 100, 100] # [24, 500, 100, 100]
pred = []

for i in range (151, 241): # Next 241 - ...
    if i % 30 != 0: # Yang boleh Genap
        continue
    df_coba2 = pd.read_csv(f'EMD_-{lim-i}.csv')
    df_coba2.set_index(pd.DatetimeIndex(df_coba2['Date']), inplace=True)
    df_coba2.drop(['Date'],axis=1,inplace=True)
    bar, kol = df_coba2.shape
    all_pred = np.zeros(30) # dua elemen karena horizon 2
    
    for j in range (0, kol):
        imfs_x = df_coba2[f'{j+1}'].values
        train, test = train_test_split(imfs_x, n_test)
        
        n_input = n_seq * n_steps
        # prepare data
        data = series_to_supervised(train, n_input)
        train_x, train_y = data[:, :-30], data[:, -30:]
        train_x = train_x.reshape((train_x.shape[0], n_seq, n_steps, 1))
        
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu', input_shape=(None,n_steps,1))))
        model.add(TimeDistributed(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu')))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(n_nodes, activation='relu'))
        model.add(Dense(n_nodes, activation='relu'))
        model.add(Dense(30))
        model.compile(loss='mse', optimizer='adam')
        
        model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
        
        n_input = n_seq * n_steps
        # prepare data
        x_input = train[-n_input:].reshape((1, n_seq, n_steps, 1))
        # forecast
        yhat = model.predict(x_input, verbose=0)
        
        for k in range(0, n_steps_out):
            all_pred[k] += list(yhat[0])[k]
                   
    for l in range(0, n_steps_out):
        pred.append(all_pred[l])
    print(' > proses ke %i' % i)

print('it took: ', (time.time()-start)*1000)
pred  

 > proses ke 180
 > proses ke 210
 > proses ke 240
it took:  11113534.916639328


[65.1558141708374,
 64.58238529786468,
 64.58931565284729,
 65.23917680233717,
 62.2209575176239,
 62.29858409613371,
 61.323313139379025,
 60.399247501045465,
 60.75757183134556,
 60.25626528635621,
 59.44583932310343,
 59.98326149582863,
 58.09822350740433,
 57.73024907708168,
 57.516500771045685,
 58.840333104133606,
 57.9153915643692,
 61.115727201104164,
 60.21476040035486,
 62.390634877607226,
 62.00281857699156,
 61.984815806150436,
 62.36475272476673,
 61.12098631262779,
 61.51688626408577,
 63.46140372753143,
 61.98245024681091,
 63.05191770195961,
 62.014352943748236,
 62.145358726382256,
 57.155184864997864,
 57.44527477025986,
 59.08540799841285,
 62.552589021623135,
 60.14215449243784,
 64.01597449183464,
 60.86125020310283,
 61.91910798847675,
 63.55768948420882,
 63.78058680891991,
 63.34731477499008,
 61.023714900016785,
 62.20454188808799,
 64.53943485021591,
 63.4983719997108,
 61.89927101135254,
 61.62578731775284,
 61.88602030277252,
 59.6342359483242,
 61.041503846

In [40]:
'''lim = 920
i = 60
j = 1
n_test = 30

n_seq = 1
n_steps = 50

df_coba2 = pd.read_csv(f'EMD_-{lim-i}.csv')
df_coba2.set_index(pd.DatetimeIndex(df_coba2['Date']), inplace=True)
df_coba2.drop(['Date'],axis=1,inplace=True)
bar, kol = df_coba2.shape
all_pred = np.zeros(30)

imfs_x = df_coba2[f'{j+1}'].values
train, test = train_test_split(imfs_x, n_test)
        
n_input = n_seq * n_steps
# prepare data
data = series_to_supervised(train, n_input)
train_x, train_y = data[:, :-30], data[:, -30:]
train_x = train_x.reshape((train_x.shape[0], n_seq, n_steps, 1))
    
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu', input_shape=(None,n_steps,1))))
model.add(TimeDistributed(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(n_nodes, activation='relu'))
model.add(Dense(n_nodes, activation='relu'))
model.add(Dense(30))
model.compile(loss='mse', optimizer='adam')

model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)'''

In [37]:
#len(train_x[0][0][0])

1

In [41]:
n_input = n_seq * n_steps
# prepare data
x_input = train[-n_input:].reshape((1, n_seq, n_steps, 1))
# forecast
yhat = model.predict(x_input, verbose=0)

In [43]:
# yhat

In [ ]:
data = series_to_supervised(train, n_input)
train_x, train_y = data[:, :-30], data[:, -30:]
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))

In [ ]:
x_input = train[-n_steps:].reshape((1, n_steps))

In [25]:
# INGAT NEXT adalah
# Pengambilan s/d 121 - 151 ; len harusnya 5, 10 atau 15; total sampai 246 - 251, s/d Proses 50
# Proses 24 ke 30 
for k in range(0, len(pred)):
    pred_all.append(pred[k])
len(pred_all)

240

In [28]:
# Simpan File tiap 50
lim = 900
i = 240
df_coba3 = pd.DataFrame(df_Br.index[-900:-660].values, columns=['Date'])
df_coba3['pred'] = pred_all
df_coba3.set_index(pd.DatetimeIndex(df_coba3['Date']), inplace=True)
df_coba3.drop(['Date'],axis=1,inplace=True)
df_coba3.to_csv(f'Pred_CNN-LSTM30_EMD-{lim}-{lim-i}.csv',index=True)
df_coba3.to_excel(f'Pred_CNN-LSTM30_EMD-{lim}-{lim-i}.xlsx',sheet_name='Sheet1',index=True)

In [ ]:
# 04. Hitung Error Measurement

In [26]:
def timeseries_evaluation_metrics_func(y_true, y_pred):    
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

In [27]:
timeseries_evaluation_metrics_func(df_Br['Close'][-900:-660], pred_all)

Evaluation metric results:-
MSE is : 18.022224271848852
MAE is : 3.057917161804158
RMSE is : 4.245259034717299
MAPE is : 4.640917662509233
R2 is : 0.09528119496285947

